# Customized Image Classification using AutoGluon

In this tutorial, we load images and the corresponding labels into [AutoGluon](https://autogluon.mxnet.io/index.html) and use this data to obtain a neural network that can classify new images. This is different from traditional machine learning where we need to manually define the neural network and then specify the hyperparameters in the training process. Instead, with just a single call to AutoGluon’s fit function, AutoGluon automatically trains many models with different hyperparameter configurations and returns the model that achieved the highest level of accuracy.

Note: Please use **GPU** for training. CPU training will lead to an unceasing running script. 

In [1]:
## make sure that `mxnet-cu101` (for GPU) shows here, rather than `mxnet` (for CPU)
!pip list | egrep mxnet

autogluon.mxnet                    0.1.0
aws-mxnet-cu101mkl                 1.6.0
keras-mxnet                        2.2.4.2


In [2]:
## install autogluon
!pip install -q autogluon
!pip list | egrep autogluon

autogluon                          0.1.0
autogluon-contrib-nlp              0.0.1b20210201
autogluon.core                     0.1.0
autogluon.extra                    0.1.0
autogluon.features                 0.1.0
autogluon.mxnet                    0.1.0
autogluon.tabular                  0.1.0
autogluon.text                     0.1.0
autogluon.vision                   0.1.0


Let's import the ImagePredictor

In [3]:
import autogluon.core as ag
from autogluon.vision import ImagePredictor

To use AutoGluon for computer vision task training, we need to organize our data with the following structure:

    data/
    ├── train/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...
    ├── test/
        ├── class1/
        ├── class2/
        ├── class3/
        ├── ...

Here each subfolder contains all images that belong to that category, e.g., `class1` contains all images belonging to the first class. We generally recommend at least 100 training images per class for reasonable classification performance, but this might depend on the type of images in your specific use-case.

## Download the dataset

For demonstration purposes, we use a subset of the [Shopee-IET](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/data) dataset from Kaggle. Each image in this data depicts a clothing item and the corresponding label specifies its clothing category. Our subset of the data contains the following possible labels: BabyPants, BabyShirt, womencasualshoes, womenchiffontop.

We download the data subset and create training/test dataset folders like below. If you use this on your own dataset, just point it to your training or test folder. Example: `train_dataset = ImagePredictor.Dataset.from_folder('mydataset/train')`

In [4]:
path = 'https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip'
train_dataset, _, test_dataset = ImagePredictor.Dataset.from_folders(path)

data/
├── test/
└── train/


Let's print the training dataset.

In [5]:
print(train_dataset)

                                                 image  label
0    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
1    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
2    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
3    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
4    /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      0
..                                                 ...    ...
795  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
796  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
797  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
798  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3
799  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...      3

[800 rows x 2 columns]


## Use AutoGluon to Fit Models

Now, let's fit a __classifier__ using AutoGluon [predictor.fit()](https://auto.gluon.ai/stable/tutorials/image_prediction/beginner.html). Within fit, the dataset is __automatically__ split into training and validation sets. The model with the best hyperparameter configuration is selected based on its performance on the __validation set__.

In [6]:
predictor = ImagePredictor()

time_limit = 10 * 60 # how long fit() should run (in seconds)
predictor.fit(train_dataset,
              nets=['ResNet50_v1b', 'ResNet18_v1b'], # default
              epochs=10,
              time_limit=time_limit,
              ngpus_per_trial=1,
              verbose=True)

INFO:gluoncv.auto.tasks.image_classification:Randomly split train_data into train[736]/validation[64] splits.
INFO:gluoncv.auto.tasks.image_classification:Starting fit without HPO
INFO:ImageClassificationEstimator:modified configs(<old> != <new>): {
INFO:ImageClassificationEstimator:root.valid.batch_size 128 != 16
INFO:ImageClassificationEstimator:root.train.rec_train ~/.mxnet/datasets/imagenet/rec/train.rec != auto
INFO:ImageClassificationEstimator:root.train.lr        0.1 != 0.01
INFO:ImageClassificationEstimator:root.train.rec_val   ~/.mxnet/datasets/imagenet/rec/val.rec != auto
INFO:ImageClassificationEstimator:root.train.batch_size 128 != 16
INFO:ImageClassificationEstimator:root.train.data_dir  ~/.mxnet/datasets/imagenet != auto
INFO:ImageClassificationEstimator:root.train.epochs    10 != 15
INFO:ImageClassificationEstimator:root.train.rec_val_idx ~/.mxnet/datasets/imagenet/rec/val.idx != auto
INFO:ImageClassificationEstimator:root.train.num_training_samples 1281167 != -1
INFO:Im

INFO:ImageClassificationEstimator:[Epoch 10] speed: 44 samples/sec	time cost: 23.684650
INFO:ImageClassificationEstimator:[Epoch 10] validation: top1=0.973750 top5=1.000000
INFO:ImageClassificationEstimator:[Epoch 10] Current best top-1: 0.973750 vs previous 0.967500, saved to /home/ec2-user/SageMaker/MLA-CV-Content/notebooks/day_2/392b15c0/.trial_0/best_checkpoint.pkl
INFO:ImageClassificationEstimator:Pickled to /home/ec2-user/SageMaker/MLA-CV-Content/notebooks/day_2/392b15c0/.trial_0/best_checkpoint.pkl
INFO:ImageClassificationEstimator:Epoch[11] Batch [49]	Speed: 46.075650 samples/sec	accuracy=0.831250	lr=0.010000
INFO:ImageClassificationEstimator:[Epoch 11] training: accuracy=0.831250
INFO:ImageClassificationEstimator:[Epoch 11] speed: 45 samples/sec	time cost: 23.619562
INFO:ImageClassificationEstimator:[Epoch 11] validation: top1=0.976250 top5=1.000000
INFO:ImageClassificationEstimator:[Epoch 11] Current best top-1: 0.976250 vs previous 0.973750, saved to /home/ec2-user/SageMaker

## Model Results

Autogluon also provides the training results, which can be accessed by calling `predictor.fit_summary()`. 

In [7]:
fit_result = predictor.fit_summary()

In [8]:
fit_result

{'train_acc': 0.875,
 'valid_acc': 0.9825,
 'total_time': 388.5355656147003,
 'best_config': {'model': 'resnet50_v1b',
  'lr': 0.01,
  'num_trials': 1,
  'epochs': 15,
  'batch_size': 16,
  'nthreads_per_trial': 128,
  'ngpus_per_trial': 1,
  'time_limits': 600,
  'search_strategy': 'random',
  'dist_ip_addrs': None,
  'log_dir': '/home/ec2-user/SageMaker/MLA-CV-Content/notebooks/day_2/392b15c0',
  'num_workers': 4,
  'gpus': [0],
  'seed': 311,
  'final_fit': False,
  'estimator': gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator,
  'wall_clock_tick': 1614817101.1394138},
 'fit_history': {'train_acc': 0.875,
  'valid_acc': 0.9825,
  'total_time': 388.5355656147003,
  'best_config': {'model': 'resnet50_v1b',
   'lr': 0.01,
   'num_trials': 1,
   'epochs': 15,
   'batch_size': 16,
   'nthreads_per_trial': 128,
   'ngpus_per_trial': 1,
   'time_limits': 600,
   'search_strategy': 'random',
   'dist_ip_addrs': None,
   'log_dir': '/home/ec2-use

We can access certain results from this summary. For example, training and validation accuracies below.

In [9]:
print('Train acc: %.3f, val acc: %.3f' %(fit_result['train_acc'], fit_result['valid_acc']))

Train acc: 0.875, val acc: 0.983


The best model and optimum hyperparameters: Learning rate, batch size, epochs can be printed with this:

In [10]:
fit_result['fit_history']['best_config']

{'model': 'resnet50_v1b',
 'lr': 0.01,
 'num_trials': 1,
 'epochs': 15,
 'batch_size': 16,
 'nthreads_per_trial': 128,
 'ngpus_per_trial': 1,
 'time_limits': 600,
 'search_strategy': 'random',
 'dist_ip_addrs': None,
 'log_dir': '/home/ec2-user/SageMaker/MLA-CV-Content/notebooks/day_2/392b15c0',
 'num_workers': 4,
 'gpus': [0],
 'seed': 311,
 'final_fit': False,
 'estimator': gluoncv.auto.estimators.image_classification.image_classification.ImageClassificationEstimator,
 'wall_clock_tick': 1614817101.1394138}

## Making Predictions

We can call the predict function to run on different images.

In [11]:
image_path = test_dataset.iloc[0]['image']
predictor.predict(image_path)

,class,score,id
0,BabyShirt,0.522461,1


Let's get predictions on the test set.

In [12]:
pred = predictor.predict(test_dataset)
print(pred)

               class     score  id  \
0          BabyShirt  0.522461   1   
1          BabyShirt  0.957889   1   
2   womencasualshoes  0.561329   2   
3          BabyPants  0.869628   0   
4          BabyPants  0.905809   0   
..               ...       ...  ..   
75   womenchiffontop  0.975787   3   
76   womenchiffontop  0.650768   3   
77   womenchiffontop  0.995997   3   
78   womenchiffontop  0.983135   3   
79   womenchiffontop  0.951276   3   

                                                image  
0   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
1   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
2   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
3   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
4   /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
..                                                ...  
75  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
76  /home/ec2-user/.gluoncv/datasets/shopee-iet/da...  
77  /home/ec2-user/.gluoncv/datasets/sh